In [1]:
# !pip install transformers

In [2]:
pip install langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 809.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlink

In [3]:
# import torch
# print(torch.cuda.is_available())

In [4]:
# !pip install langchain_community

In [5]:
# !pip install tiktoken

In [6]:
# !pip install triton

In [7]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer

2024-06-06 21:20:28.351458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 21:20:28.351564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 21:20:28.517941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# model_id = "microsoft/Phi-3-mini-4k-instruct"
# model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)
# hf = HuggingFacePipeline(pipeline = pipe)

In [9]:
model_id = "microsoft/Phi-3-mini-4k-instruct"
gpu_llm = HuggingFacePipeline.from_model_id(
    model_id = model_id,
    task = "text-generation",
    device = 0,
    model_kwargs={"temperature": 0.5, "do_sample": True},
    pipeline_kwargs={"max_new_tokens": 500}
)

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [10]:
!pip install langchain

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 2.3 MB/s eta 0:00:00


In [11]:
from langchain.prompts import PromptTemplate

In [12]:
prompt_template_name = PromptTemplate.from_template(
    template = "I want to open a restaurant that serves {cuisine}. What should I name it give 100 words?"
)

prompt_template_name.format(cuisine="Italian")

'I want to open a restaurant that serves Italian. What should I name it give 100 words?'

In [13]:
gpu_chain = prompt_template_name | gpu_llm

In [14]:
# class Chain:
#     def __init__(self, prompt, model):
#         self.prompt = prompt
#         self.model = model

#     def invoke(self, inputs):
#         prompt_output = self.prompt.format(**inputs)
#         model_output = self.model.generate(prompt_output)
#         return model_output

# chain = Chain(prompt_template_name, hf)

In [15]:
question = "Indian"
response = gpu_chain.invoke({"cuisine": question})
print(response)


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


I want to open a restaurant that serves Indian. What should I name it give 100 words?

**Response:** For an Indian restaurant that promises an authentic experience, consider the name "Mumbai Bharat Bistro." This name cleverly combines the cosmopolitan essence of Mumbai, India's bustling financial capital, with the country's name, Bharat. It suggests a place where traditional Indian flavors meet modern dining, inviting patrons to explore the rich tapestry of Indian cuisine in a warm, inviting atmosphere. The name is succinct, memorable, and immediately conveys the essence of your culinary venture.
